In [7]:
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib import style
style.use('seaborn-talk')
import numpy as np
import pandas as pd
from scipy.fftpack import fft
import glob

In [2]:
outpath = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\chunking'
figpath = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\chunking\figures'
source = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Centroids_Final\Raw_NG'

In [12]:
#sample data such that there are no NaNs within each path and that each path is greater than 100 frames (fft cannot tolerate NaNs, is removing better than smoothening?)
def removeNaN(df): 
    if np.any(df.X_position.isnull()):
        a = np.asarray(np.where(df.X_position.isnull())).ravel()
        b = a[1:]
        c = a[:-1]
        d = b-c
        last_chunk = len(df.X_position) - a[-1]


        if last_chunk > 100:

            start = a[-1]+1
            end = a[-1] + last_chunk-1

        elif a[0] > 100:
            start = 0
            end = a[0]-1


        elif d.max() > 100:
            e = d.argmax()
            start = a[e]+1
            end = a[e+1]-1

        else:
            print('error')
    
        time = df.FrameNumber.values[start:end]
        x_no_nan = df.X_position.values[start:end]
        y_no_nan = df.Y_position.values[start:end]
        return(x_no_nan, y_no_nan,  time)
    else:
        x = df.X_position.values
        y = df.Y_position.values
        time = df.FrameNumber.values
        return(x,y, time)

In [5]:
def Tortuosity(x, time):
    
    
    t = time/100
    
    Nx = len(x)  #number of points
    Nt = Nx
    Time = t[-1]-t[0]   #total sample time (in seconds)c
    fs = Nx/Time   # sample frequency
    TT = 1/fs  #delta t.
 
    meanx = np.mean(x)
    
    # . fftxvel is complex with real and imaginary part.
    #Nyquist spacing (first half of Fourier) -- the list of frequencies for which we have Fourier coefficients
    fftx = fft(x-meanx)    # this is the Fourier tranform of the x velocity
    xf = np.linspace(0.0, 1.0/(2.0*TT), Nx//2)  

    
    return xf, Nx, fftx

In [8]:
csv_list = glob.glob(source + '\*Centroid*.csv')
print(len(csv_list))

52


In [1]:
name_list = []
xf_list = []
Nx_list = []
fft_Cxvel_list = []
stdxv_list = []
meanxv_list = []
turb_intens_list = []
conditioned_xvel_list = []
xvel = []
yvel = []
t = []
window = []
visit_number_list = []
f, ax = plt.subplots(1,1, figsize = (20,10), sharex = True, sharey = True)

for element in csv_list[:4]:
    df = pd.read_csv(element)
    x, y, time = removeNaN(df)
    
    visit_number = df.visit_number[0]
    
    xf, Nx, fftx = Tortuosity(x, time)
    yf, Ny, ffty = Tortuosity(y, time)

    name_list.append(element[81:-4])
    
#     xf_list.append(xf)
#     Nx_list.append(Nx)
#     fft_Cxvel_list.append(fft_Cxvel)
#     stdxv_list.append(stdxv)
#     meanxv_list.append(meanxv)
#     turb_intens_list.append(turb_intens)
#     conditioned_xvel_list.append(conditioned_xvel)
#     visit_number_list.append(visit_number)
    
    
    sum_xy = (2.0/Ny * np.abs(ffty[0:Ny//2]) + 2.0/Nx * np.abs(fftx[0:Nx//2]))
    ax.plot(yf, sum_xy, "-o")
    ax.set_ylim(0,300)
    
#     fig.title(element[81:-4])
#     plt.savefig(figpath + "\\" + element[81:-4] + '_x.png')
#     plt.show()   
#     sb.set()

NameError: name 'plt' is not defined

In [14]:
name_list_ = []
for name in name_list:
    if name.startswith('L0.1_c-3_m'):
        name = name[:12]
        if name.endswith('_'):
            name = name[:-1]
        
    elif name.startswith('Sham'):
        name = name[:7]
        if name.endswith('_'):
            name = name[:-1]
    elif name.startswith('Experimental'):
        name = name[:15]
        if name.endswith('_'):
            name = name[:-1]
    
    name_list_.append(name)
#     print(name)
name_list = name_list_
name_list

['Experimental_1',
 'Experimental_1',
 'Experimental_1',
 'Experimental_2',
 'Experimental_3',
 'Experimental_4',
 'Experimental_4',
 'Experimental_5',
 'Experimental_7',
 'Experimental_7',
 'Experimental_7',
 'Experimental_7',
 'Sham_10',
 'Sham_11',
 'Sham_14',
 'Sham_14',
 'Sham_15',
 'Sham_16',
 'Sham_17',
 'Sham_1',
 'Sham_1',
 'Sham_1',
 'Sham_1',
 'Sham_1',
 'Sham_2',
 'Sham_2',
 'Sham_3',
 'Sham_3',
 'Sham_3',
 'Sham_4',
 'Sham_4',
 'Sham_4',
 'Sham_6',
 'Sham_6',
 'Sham_6',
 'Sham_6',
 'Sham_6',
 'Sham_7',
 'Sham_7',
 'Sham_7',
 'Sham_7',
 'Sham_7',
 'Sham_7',
 'Sham_7']

In [15]:
len(name_list), len(Nx_list), len(name_list), len(stdxv_list), len(meanyv_list), len(turb_intens_list)

(44, 44, 44, 44, 44, 44)

In [16]:
condition = ["Experimental"]*12
condition.extend(["Sham"]*32)
new_df = (pd.DataFrame({'name' : name_list, 'Nx' : Nx_list,
                            'stdxv': stdxv_list, 'meanxv' : meanxv_list
                       ,'turb_intens' : turb_intens_list, 'condition': condition, 'visit_number': visit_number_list}))
other_df = (pd.DataFrame({'xf' : xf_list, 'fft_Cxvel': fft_Cxvel_list, 'filterd_xvel' : conditioned_xvel_list}))



In [17]:
new_y_df = (pd.DataFrame({'name' : name_list, 'Ny' : Ny_list,
                            'stdyv': stdyv_list, 'meanyv' : meanyv_list
                       ,'turb_intens' : turb_y_intens_list, 'condition': condition, 'visit_number': visit_number_ylist}))
other_y_df = (pd.DataFrame({'yf' : yf_list, 'fft_Cyvel': fft_Cyvel_list, 'filterd_yvel' : conditioned_yvel_list}))

In [18]:
new_df.to_csv(r"C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\bad_videos_removed" + r"\Tortuosity_Single_removed.csv")
other_df.to_csv(r"C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\bad_videos_removed" + r"\tortuosity_Longer.csv")

In [19]:
new_y_df.to_csv(r"C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\bad_videos_removed" + r"\Tortuosity_y_Single_removed.csv")
other_y_df.to_csv(r"C:\Users\Daniellab\Desktop\Jorge_Analysis\Tortuosity\bad_videos_removed" + r"\tortuosity_y_Longer.csv")

In [20]:
df = pd.read_csv(outpath + "\Tortuosity_Single.csv")

name_list = df.name


name_list

0     Experimental_1
1     Experimental_1
2     Experimental_1
3     Experimental_2
4     Experimental_2
5     Experimental_3
6     Experimental_4
7     Experimental_4
8     Experimental_5
9     Experimental_5
10    Experimental_7
11    Experimental_7
12    Experimental_7
13    Experimental_7
14           Sham_10
15           Sham_11
16           Sham_11
17           Sham_14
18           Sham_14
19           Sham_15
20           Sham_15
21           Sham_15
22           Sham_15
23           Sham_15
24           Sham_16
25           Sham_16
26           Sham_17
27           Sham_18
28            Sham_1
29            Sham_1
30            Sham_1
31            Sham_1
32            Sham_1
33            Sham_1
34            Sham_1
35            Sham_2
36            Sham_2
37            Sham_2
38            Sham_3
39            Sham_3
40            Sham_3
41            Sham_4
42            Sham_4
43            Sham_4
44            Sham_6
45            Sham_6
46            Sham_6
47           

In [21]:
df_y = pd.read_csv(outpath + "\Tortuosity_y_Single.csv")


name_y_list = df_y.name

print(len(name_y_list))

57
